In [9]:
import pandas as pd
import numpy as np

from causallearn.search.ConstraintBased.PC import pc

In [10]:
# Generate data where X -> Y <- Z
X = np.random.uniform(size=10000)
eps = np.random.normal(size=10000)
delta = np.random.uniform(size=10000)
Y = -7*X + 0.5*delta
Z = 2*X + Y + eps

# Create DataFrame with named variables
data = pd.DataFrame({'X': X, 'Y': Y, 'Z': Z})

In [11]:
# Store original column names
variable_names = list(data.columns)

g = pc(data.to_numpy())

Depth=1, working on node 2: 100%|██████████| 3/3 [00:00<00:00, 292.91it/s]


In [12]:
for i, node in enumerate(g.G.nodes):
    node.name = variable_names[i]

In [13]:
g.G.graph

array([[ 0, -1, -1],
       [-1,  0, -1],
       [-1, -1,  0]])

In [ ]:
edges = []
indices = np.where(g.G.graph != 0)
processed_pairs = set()

for i, j in zip(indices[0], indices[1]):
    
    node_pair = frozenset([i.item(), j.item()])
    
    if node_pair in processed_pairs:
        continue
        
    if g.G.graph[i,j] == 1 and g.G.graph[j,i] == -1:
        edges.append({
            'from': i.item(),
            'to': j.item(),
            'type': "->"
        })
    
    elif g.G.graph[i,j] == -1 and g.G.graph[j,i] == -1:
        edges.append({
            'from': i.item(),
            'to': j.item(),
            'type': "--"
        })
        processed_pairs.add(node_pair) 
    
    elif g.G.graph[i,j] == 1 and g.G.graph[j,i] == 1:
        edges.append({
            'from': i.item(),
            'to': j.item(),
            'type': "<->"
        })
        processed_pairs.add(node_pair)

In [15]:
edges

[{'from': 0, 'to': 1, 'type': '--'},
 {'from': 0, 'to': 2, 'type': '--'},
 {'from': 1, 'to': 2, 'type': '--'}]